# Chocolate Rating Analysis

## Web scraping data to then process, clean, etc. in the notebook.

In [4]:
# import warnings
# warnings.filterwarnings('ignore')

# from requests import get
# from bs4 import BeautifulSoup
# import pandas as pd
# from selenium import webdriver
# import chromedriver_autoinstaller
# def create_driver(headless=True):
#     driver = chromedriver_autoinstaller.install(cwd=True)
#     chrome_options = webdriver.ChromeOptions()     
#     if headless:
#         chrome_options.add_argument("--headless")
#         driver = webdriver.Chrome(driver, 
#                               chrome_options = chrome_options)
#         return driver
# driver = create_driver()

In [2]:
# driver.get("http://flavorsofcacao.com/chocolate_database.html")

In [3]:
# div_id = "spryregion1"

# html = driver.page_source
# soup = BeautifulSoup(html)
# div = soup.find('div', {'id':div_id})
# div

# table = div.find('table')
# table


In [4]:
# pd.read_html(str(table))[0]

In [7]:
# df.to_csv('../../../data/chocolate.csv', index=False)

## Importing Necessary Python Libraries 

In [318]:
import pandas as pd 
import numpy as np 
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.impute import SimpleImputer
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import plot_confusion_matrix, auc, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from imblearn.over_sampling import SMOTE

all_data = pd.read_csv('../../../data/chocolate.csv')
all_data.head()

# new data frame with split value columns 


,REF,Company (Manufacturer),Company Location,Review Date,Country of Bean Origin,Specific Bean Origin or Bar Name,Cocoa Percent,Ingredients,Most Memorable Characteristics,Rating
0,2454,5150,U.S.A.,2019,Madagascar,"Bejofo Estate, batch 1",76%,"3- B,S,C","cocoa, blackberry, full body",3.75
1,2458,5150,U.S.A.,2019,Dominican Republic,"Zorzal, batch 1",76%,"3- B,S,C","cocoa, vegetal, savory",3.50
2,2454,5150,U.S.A.,2019,Tanzania,"Kokoa Kamili, batch 1",76%,"3- B,S,C","rich cocoa, fatty, bready",3.25
3,797,A. Morin,France,2012,Peru,Peru,63%,"4- B,S,C,L","fruity, melon, roasty",3.75
4,797,A. Morin,France,2012,Bolivia,Bolivia,70%,"4- B,S,C,L","vegetal, nutty",3.50


## Data Cleaning 

### Replacing column spaces with underscores and lower casing column names for simplicity when calling them. 

In [319]:
all_data.columns = all_data.columns.str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.lower()

all_data = all_data.replace("%", "", regex=True)
all_data

,ref,company_manufacturer,company_location,review_date,country_of_bean_origin,specific_bean_origin_or_bar_name,cocoa_percent,ingredients,most_memorable_characteristics,rating
0,2454,5150,U.S.A.,2019,Madagascar,"Bejofo Estate, batch 1",76,"3- B,S,C","cocoa, blackberry, full body",3.75
1,2458,5150,U.S.A.,2019,Dominican Republic,"Zorzal, batch 1",76,"3- B,S,C","cocoa, vegetal, savory",3.50
2,2454,5150,U.S.A.,2019,Tanzania,"Kokoa Kamili, batch 1",76,"3- B,S,C","rich cocoa, fatty, bready",3.25
3,797,A. Morin,France,2012,Peru,Peru,63,"4- B,S,C,L","fruity, melon, roasty",3.75
4,797,A. Morin,France,2012,Bolivia,Bolivia,70,"4- B,S,C,L","vegetal, nutty",3.50
...,...,...,...,...,...,...,...,...,...,...
2357,1205,Zotter,Austria,2014,Blend,Raw,80,"4- B,S*,C,Sa","waxy, cloying, vegetal",2.75
2358,1996,Zotter,Austria,2017,Colombia,"APROCAFA, Acandi",75,"3- B,S,C","strong nutty, marshmallow",3.75
2359,2170,Zotter,Austria,2018,Belize,Maya Mtn,72,"3- B,S,C","muted, roasty, accessible",3.50
2360,2170,Zotter,Austria,2018,Congo,Mountains of the Moon,70,"3- B,S,C","fatty, mild nuts, mild fruit",3.25


In [320]:
df =  all_data.copy()

In [321]:
# retreiving rows from the third index and on - leaving out rows with empty values
df['ingredient_list'] = [np.nan if l is np.nan else l[3:] for l in df['ingredients']]

In [322]:
df['ingredient_list'].value_counts()

B,S,C          903
B,S            661
B,S,C,L        273
B,S,C,V,L      184
B,S,C,V        139
B,S*            31
B,S*,C,Sa       20
B,S*,C          12
B,S,L            8
B,S*,C,V         7
,S,C,V,Sa        6
B,S,C,Sa         5
B,S,V,L          5
B                5
,S,C,V,L,Sa      4
B,S*,V,L         3
B,S,V            3
B,S*,C,L         2
B,S,C,L,Sa       1
B,C              1
B,S*,Sa          1
Name: ingredient_list, dtype: int64

In [323]:
# leaving out occurances below 8 

most_common = (df['ingredient_list'].value_counts()).iloc[:9].index


df = df[df['ingredient_list'].isin(most_common)]

### Clarifying important predictor values (x) for the target value (y).

In [326]:
features = df.drop(['rating',  'ingredients', 'review_date', 'ref'], axis =1)
X = features
y = df.rating



### Splitting the data into three subsets of training and validation data for the future models.
    Two train test splits create three subsets of the original dataset which allows for the training data to not be bled into the test data - this reduced model's bias towards the pre-existing testing data, thus assuring maximum performance on future test sets which the model has never been exposed. 

In [327]:
#performing train test split for test set (subsets 1/3)
X_tr, X_test, y_tr, y_test = train_test_split(X, y, test_size=.15, random_state=42)


#performing a train test split for train and validation set (subsets - 3/3)
X_train, X_val, y_train, y_val = train_test_split(X_tr, y_tr, test_size=.15, random_state=42) 

In [328]:
#checking to ensure the shape of the columns and rows are still the same for the X and y values after the train test split.
X_tr.shape, y_tr.shape

((1896, 7), (1896,))

### Replacing any existing missing values  

In [329]:
#ingredients have 88 rows that have no imputs.
df.isna().sum()

ref                                 0
company_manufacturer                0
company_location                    0
review_date                         0
country_of_bean_origin              0
specific_bean_origin_or_bar_name    0
cocoa_percent                       0
ingredients                         0
most_memorable_characteristics      0
rating                              0
ingredient_list                     0
dtype: int64

In [330]:
# the empty value replacement will be done using the 
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

In [331]:
imputer.fit(X_train)

SimpleImputer(strategy='most_frequent')

In [332]:
X_train_imputed = pd.DataFrame(imputer.transform(X_train), columns=X.columns)
X_val_imputed = pd.DataFrame(imputer.transform(X_val), columns=X.columns)

In [333]:
print("X_train missing: ", X_train_imputed.isna().sum().sum())
print('X_val missing: ', X_val_imputed.isna().sum().sum())

X_train missing:  0
X_val missing:  0


In [334]:
# WHY DO WE USE  X INSTEAD OF X_TRAIN  

In [346]:
#num_cols_train = list(X.select_dtypes('number').columns)
#num_cols_train

In [347]:
cat_cols_train = list(X.select_dtypes('object').columns)
cat_cols_train

['company_manufacturer',
 'company_location',
 'country_of_bean_origin',
 'specific_bean_origin_or_bar_name',
 'cocoa_percent',
 'most_memorable_characteristics',
 'ingredient_list']

In [348]:
#num_cols_val = list(X.select_dtypes('number').columns)
#num_cols_val

In [349]:
cat_cols_val = list(X.select_dtypes('object').columns)
cat_cols_val

['company_manufacturer',
 'company_location',
 'country_of_bean_origin',
 'specific_bean_origin_or_bar_name',
 'cocoa_percent',
 'most_memorable_characteristics',
 'ingredient_list']

In [350]:
# Instantiate StandardScaler
scaler = StandardScaler()

In [351]:
# fit on Imputed training data with numeric columns in train
#scaler.fit(X_train_imputed[num_cols_train])

In [352]:
# fit on Imputed training data with numeric  columns in val
#scaler.fit(X_train_imputed[num_cols_val])

In [354]:
# transform training and validation data
#X_train_scaled = pd.DataFrame(scaler.transform(X_train_imputed[num_cols_train]), columns=num_cols)
#X_val_scaled = pd.DataFrame(scaler.transform(X_val_imputed[num_cols_val]), columns=num_cols)

In [355]:
#X_train_scaled

In [356]:
ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')

In [362]:
# fit on training categorical data
ohe.fit(X_train_imputed[['ingredient_list']])

OneHotEncoder(handle_unknown='ignore', sparse=False)

In [305]:
# fit on training categorical data
ohe.fit(X_train_imputed[cat_cols_train])

OneHotEncoder(handle_unknown='ignore', sparse=False)

In [306]:
# fit on validation categorical data
ohe.fit(X_val_imputed[cat_cols_val])

OneHotEncoder(handle_unknown='ignore', sparse=False)

In [365]:
X_train_encoded = ohe.transform(X_train_imputed[['ingredient_list']], )
X_val_encoded = ohe.transform(X_val_imputed[['ingredient_list']])

In [307]:
# transform training and validation data
X_train_encoded = ohe.transform(X_train_imputed[cat_cols_train], )
X_val_encoded = ohe.transform(X_val_imputed[cat_cols_val])

In [366]:
X_train_encoded_df = pd.DataFrame(X_train_encoded, columns=ohe.get_feature_names())
X_val_encoded_df = pd.DataFrame(X_val_encoded, columns=ohe.get_feature_names())
X_train_encoded_df

,"x0_B,S","x0_B,S*","x0_B,S*,C","x0_B,S*,C,Sa","x0_B,S,C","x0_B,S,C,L","x0_B,S,C,V","x0_B,S,C,V,L","x0_B,S,L"
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
1606,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1607,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1608,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1609,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [367]:
X_train_encoded_df.loc[:].sum()

x0_B,S          461.0
x0_B,S*          20.0
x0_B,S*,C         8.0
x0_B,S*,C,Sa     11.0
x0_B,S,C        656.0
x0_B,S,C,L      212.0
x0_B,S,C,V      102.0
x0_B,S,C,V,L    136.0
x0_B,S,L          5.0
dtype: float64

In [269]:
X_train_df = pd.concat([X_train_scaled, X_train_encoded_df], axis=1)
X_val_df = pd.concat([X_val_scaled, X_val_encoded_df], axis=1)

In [270]:
X_train_df.shape

(1611, 844)

In [271]:
X_val_df.shape

(285, 844)

In [272]:
df

,ref,company_manufacturer,company_location,review_date,country_of_bean_origin,specific_bean_origin_or_bar_name,cocoa_percent,ingredients,most_memorable_characteristics,rating,ingredient_list
0,2454,5150,U.S.A.,2019,Madagascar,"Bejofo Estate, batch 1",76,"3- B,S,C","cocoa, blackberry, full body",3.75,"B,S,C"
1,2458,5150,U.S.A.,2019,Dominican Republic,"Zorzal, batch 1",76,"3- B,S,C","cocoa, vegetal, savory",3.50,"B,S,C"
2,2454,5150,U.S.A.,2019,Tanzania,"Kokoa Kamili, batch 1",76,"3- B,S,C","rich cocoa, fatty, bready",3.25,"B,S,C"
3,797,A. Morin,France,2012,Peru,Peru,63,"4- B,S,C,L","fruity, melon, roasty",3.75,"B,S,C,L"
4,797,A. Morin,France,2012,Bolivia,Bolivia,70,"4- B,S,C,L","vegetal, nutty",3.50,"B,S,C,L"
...,...,...,...,...,...,...,...,...,...,...,...
2357,1205,Zotter,Austria,2014,Blend,Raw,80,"4- B,S*,C,Sa","waxy, cloying, vegetal",2.75,"B,S*,C,Sa"
2358,1996,Zotter,Austria,2017,Colombia,"APROCAFA, Acandi",75,"3- B,S,C","strong nutty, marshmallow",3.75,"B,S,C"
2359,2170,Zotter,Austria,2018,Belize,Maya Mtn,72,"3- B,S,C","muted, roasty, accessible",3.50,"B,S,C"
2360,2170,Zotter,Austria,2018,Congo,Mountains of the Moon,70,"3- B,S,C","fatty, mild nuts, mild fruit",3.25,"B,S,C"


## Preprocessing: